<a href="https://colab.research.google.com/github/santisf/datascience_coder/blob/main/clase_04_teorica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

## Micro challenge

Hacer un grafico de lineas para los siguientes:
- EU_Sales
- JP_Sales
- Other_Sales

*Teniendo en cuenta el axis Year_of_Release y por Developer*

In [ ]:
url_video_games:str = r"https://raw.githubusercontent.com/Andru-1987/csv_files_ds/refs/heads/main/Video_Games.csv"
video_games_df = pd.read_csv(url_video_games)


- *Completitud* : No nulls
- *Exactitud* :
    * No sean ambigüos, que sean correctos( no existan valores fuera de norma)
    * ISO 8601 --> "March 4, 2025" -> %Y-%m-%d.
    * Data duplicada() -->>> Jamas debe pasar.

In [ ]:
video_games_df.info()

In [ ]:
video_games_df.isna().sum()

# Conteo de nulls

In [ ]:
def mostrar_perc_nulls(df:pd.DataFrame) ->  pd.DataFrame:

    total_filas = len(df)
    nulos = df.isna().sum()
    percentage = (nulos / total_filas) * 100

    resumen = pd.DataFrame(
        {
            "columna": nulos.index,
            "nulos": nulos.values,
            "percentage": percentage.values.round(2)
        }
    )

    return resumen.reset_index(drop=True).sort_values(by="percentage", ascending=False)



In [ ]:
mostrar_perc_nulls(video_games_df)

*Duplicados*
- Hacer una funcion que me permita ver los datos duplicados

In [ ]:
# suponiendo que tengo dos columnas como PK's
subset_pk:list[str] = ["Name","Platform"]
video_games_df.duplicated(subset=subset_pk).sum()


In [ ]:
video_games_df.duplicated().sum()

In [ ]:
# transformar el año en tipo fecha

video_games_df["Year_of_Release"] = pd.to_datetime(video_games_df.Year_of_Release, format="%Y")


In [ ]:
# video_games_df["Year_of_Release"] =
video_games_df[video_games_df.Year_of_Release.isna()]

In [ ]:
moda_fecha = video_games_df["Year_of_Release"].mode()

In [ ]:
# video_games_df["Year_of_Release"]= video_games_df["Year_of_Release"].fillna(moda_fecha)

In [ ]:
video_games_df["Year_of_Release"] = video_games_df["Year_of_Release"].fillna(method="ffill")

In [ ]:
# video_games_df.iloc[16376]

In [ ]:
mostrar_perc_nulls(video_games_df)

In [ ]:
# video_games_df["Developer"] = video_games_df.Developer.str.lower().str.replace("-"," ")

In [ ]:
def fill_na_by_publisher(row):
    if pd.isna(row["Developer"]): # ve si es null

        if row["Publisher"] in ["Atari", "Warner Bros. Interactive Entertainment"]: # analiza si el publisher es alguna de esas opciones
            return "Nintendo" # --> entonces me coloc como nintendo
        else:
            return "Appel Inc." # --> es apple

    return row["Developer"] # --> retorna esa transformacion

In [ ]:
# reemplazo condicional
video_games_df["Developer"] = video_games_df.apply(fill_na_by_publisher, axis=1)



In [ ]:
mostrar_perc_nulls(video_games_df)

In [ ]:

## busqueda de los de mayores ventas
top_devs = (
    video_games_df.groupby("Developer")[["EU_Sales","JP_Sales","Other_Sales"]]
    .sum()
    .sum(axis=1)
    .nlargest(5)
    .index
)

In [ ]:
columnas_keep = ["Year_of_Release","Other_Sales","EU_Sales","JP_Sales","Developer"]

ventas_graficar_df = video_games_df[columnas_keep]
ventas_graficar_df = ventas_graficar_df[ventas_graficar_df["Developer"].isin(top_devs)]
ventas_graficar_df.head()

In [ ]:
ventas_graficar_df = ventas_graficar_df.set_index("Year_of_Release").sort_index()

In [ ]:
ventas_graficar_df.info()

*Graficar este challenge por los graficos de linea*

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
regiones = ["EU_Sales", "JP_Sales", "Other_Sales"]

In [ ]:
# Agrupamos por Developer y Year_of_Release
ventas_resampled =(
    ventas_graficar_df
    # un objeto de manipulacion agrupada se debe usar Grouper -->
    .groupby(["Developer", pd.Grouper(freq="YE")])[regiones]
    .sum()
    .reset_index()
)

ventas_resampled

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 9))

# Grafica iterable con regiones y la data sampleada

for ax, region in zip(axes, regiones):
    for dev, grupo in ventas_resampled.groupby("Developer"):
        ax.plot(
            grupo["Year_of_Release"],  # columna con fecha resampleada
            grupo[region],
            label=dev,
            alpha=0.7
        )
    ax.set_title(region.replace("_", " "))
    ax.set_ylabel("Millones")
    ax.grid(True, alpha=0.2)
    ax.legend(fontsize=8)

    # Girar etiquetas del eje X
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

plt.suptitle("Ventas por Región y Developer", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()
